# 🏴‍☠️ UAB THE HACK! 2025 - WiFi Dataset Analysis
## AP Time Slot Busyness Analysis

**Objetivo:** Analizar los datos de Access Points (APs) para determinar qué tan ocupado está cada intervalo de tiempo

**Dataset:**
- Access Points (APs): Archivos JSON con snapshots de APs del campus
- Cada snapshot contiene: número de clientes conectados, timestamp, y métricas de rendimiento
- Período: Abril-Julio 2025

**Métricas de "Busyness" (Ocupación):**
- Total de clientes conectados por intervalo de tiempo
- Promedio de clientes por AP por intervalo
- Número de APs activos por intervalo
- Utilización de CPU promedio por intervalo
- Identificación de horas pico

---


## 📦 1. Importar Librerías

Importamos las librerías necesarias para el análisis de intervalos de tiempo.


In [ ]:
# Librerías estándar y científicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from datetime import datetime, timedelta

# Configuración general
warnings.filterwarnings("ignore")
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams.update({
    "figure.figsize": (14, 7),
    "axes.titlesize": 16,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10
})

# Carga de funciones personalizadas
import sys
UTILS_PATH = Path('utils').resolve()
if str(UTILS_PATH) not in sys.path:
    sys.path.append(str(UTILS_PATH))
from data_loader import load_aps, print_dataset_summary

print("✅ Librerías y utilidades cargadas correctamente")


## 📂 2. Cargar Datos de Access Points

Cargamos los datos de APs. Puedes ajustar `max_files` para cargar más o menos datos.

**Nota:** Para análisis completo, usa `max_files=None` (puede tardar varios minutos)


In [ ]:
# Cargar Access Points
# Ajusta max_files según necesites: None = todos los archivos, o un número específico
df_aps = load_aps(
    data_dir="../anonymized_data/aps",
    max_files=50,  # Cambia a None para cargar todos los archivos
    verbose=True
)

print("\n" + "="*60)
print(f"🎯 APs cargados: {len(df_aps):,} registros")
print("="*60)

# Verificar que tenemos los campos necesarios
if 'timestamp' in df_aps.columns:
    print(f"\n📅 Rango temporal:")
    print(f"   Inicio: {df_aps['timestamp'].min()}")
    print(f"   Fin:    {df_aps['timestamp'].max()}")
    print(f"   Días:   {(df_aps['timestamp'].max() - df_aps['timestamp'].min()).days}")
else:
    print("⚠️  No se encontró columna 'timestamp'")


## 🔍 3. Preparar Datos para Análisis de Intervalos

Extraemos información temporal y preparamos los datos para el análisis por intervalos.


In [ ]:
# Crear columnas temporales para análisis
df_aps['date'] = df_aps['timestamp'].dt.date
df_aps['hour'] = df_aps['timestamp'].dt.hour
df_aps['day_of_week'] = df_aps['timestamp'].dt.day_name()
df_aps['day_of_week_num'] = df_aps['timestamp'].dt.dayofweek  # 0=Lunes, 6=Domingo

# Crear intervalos de tiempo (time slots)
# Opción 1: Por hora (más simple)
df_aps['time_slot_hour'] = df_aps['timestamp'].dt.floor('H')

# Opción 2: Por 15 minutos (más detallado)
df_aps['time_slot_15min'] = df_aps['timestamp'].dt.floor('15min')

# Opción 3: Por 30 minutos
df_aps['time_slot_30min'] = df_aps['timestamp'].dt.floor('30min')

print("✅ Columnas temporales creadas:")
print(f"   - time_slot_hour: {df_aps['time_slot_hour'].nunique()} intervalos únicos")
print(f"   - time_slot_15min: {df_aps['time_slot_15min'].nunique()} intervalos únicos")
print(f"   - time_slot_30min: {df_aps['time_slot_30min'].nunique()} intervalos únicos")

# Verificar campos disponibles
print("\n📋 Campos disponibles para análisis:")
print(f"   - client_count: {'✓' if 'client_count' in df_aps.columns else '✗'}")
print(f"   - cpu_utilization: {'✓' if 'cpu_utilization' in df_aps.columns else '✗'}")
print(f"   - name: {'✓' if 'name' in df_aps.columns else '✗'}")


## 📊 4. Análisis de Ocupación por Intervalos de Tiempo

Analizamos qué tan ocupado está cada intervalo de tiempo usando diferentes métricas.


In [ ]:
# Análisis por hora (time_slot_hour)
# Agrupar por intervalo de hora y calcular métricas de ocupación
hourly_stats = df_aps.groupby('time_slot_hour').agg({
    'client_count': ['sum', 'mean', 'max', 'std'],
    'name': 'nunique',  # Número de APs únicos en ese intervalo
    'cpu_utilization': 'mean' if 'cpu_utilization' in df_aps.columns else lambda x: 0
}).round(2)

# Aplanar nombres de columnas
hourly_stats.columns = ['total_clients', 'avg_clients_per_ap', 'max_clients_single_ap', 
                        'std_clients', 'unique_aps', 'avg_cpu_utilization']

# Resetear índice para tener time_slot_hour como columna
hourly_stats = hourly_stats.reset_index()

# Agregar información adicional
hourly_stats['hour'] = hourly_stats['time_slot_hour'].dt.hour
hourly_stats['date'] = hourly_stats['time_slot_hour'].dt.date
hourly_stats['day_of_week'] = hourly_stats['time_slot_hour'].dt.day_name()

print("📊 Estadísticas de ocupación por hora:")
print(hourly_stats.head(10))


### 4.1 Visualización: Total de Clientes por Hora del Día

Muestra el total de clientes conectados en cada hora del día (promedio entre todos los días).


In [ ]:
# Agrupar por hora del día (promedio entre todos los días)
hourly_avg = hourly_stats.groupby('hour').agg({
    'total_clients': 'mean',
    'avg_clients_per_ap': 'mean',
    'unique_aps': 'mean',
    'avg_cpu_utilization': 'mean'
}).reset_index()

# Crear visualización
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Gráfico 1: Total de clientes por hora
axes[0].plot(hourly_avg['hour'], hourly_avg['total_clients'], 
             marker='o', linewidth=2.5, markersize=8, color='#2c3e50', label='Total Clientes')
axes[0].fill_between(hourly_avg['hour'], hourly_avg['total_clients'], 
                      alpha=0.3, color='#3498db')
axes[0].set_title('📊 Total de Clientes Conectados por Hora del Día (Promedio)', 
                  fontsize=16, fontweight='bold', pad=15)
axes[0].set_xlabel('Hora del Día', fontsize=12)
axes[0].set_ylabel('Total de Clientes', fontsize=12)
axes[0].set_xticks(range(0, 24))
axes[0].grid(alpha=0.3, linestyle='--')
axes[0].legend(fontsize=11)

# Identificar hora pico
peak_hour = hourly_avg.loc[hourly_avg['total_clients'].idxmax(), 'hour']
peak_clients = hourly_avg['total_clients'].max()
axes[0].axvline(x=peak_hour, color='red', linestyle='--', linewidth=2, 
                label=f'Hora Pico: {int(peak_hour)}:00')
axes[0].legend(fontsize=11)

# Gráfico 2: Promedio de clientes por AP y número de APs activos
ax2_twin = axes[1].twinx()

line1 = axes[1].plot(hourly_avg['hour'], hourly_avg['avg_clients_per_ap'], 
                     marker='s', linewidth=2, markersize=7, color='#e74c3c', 
                     label='Promedio Clientes/AP')
axes[1].set_xlabel('Hora del Día', fontsize=12)
axes[1].set_ylabel('Promedio Clientes por AP', fontsize=12, color='#e74c3c')
axes[1].tick_params(axis='y', labelcolor='#e74c3c')
axes[1].set_xticks(range(0, 24))
axes[1].grid(alpha=0.3, linestyle='--')

line2 = ax2_twin.plot(hourly_avg['hour'], hourly_avg['unique_aps'], 
                      marker='^', linewidth=2, markersize=7, color='#27ae60', 
                      label='APs Activos')
ax2_twin.set_ylabel('Número de APs Activos', fontsize=12, color='#27ae60')
ax2_twin.tick_params(axis='y', labelcolor='#27ae60')

axes[1].set_title('📡 Promedio de Clientes por AP y Número de APs Activos por Hora', 
                  fontsize=16, fontweight='bold', pad=15)

# Combinar leyendas
lines = line1 + line2
labels = [l.get_label() for l in lines]
axes[1].legend(lines, labels, loc='upper left', fontsize=11)

plt.tight_layout()
plt.show()

print(f"\n🕐 Hora pico: {int(peak_hour)}:00 con {peak_clients:,.0f} clientes conectados en promedio")
print(f"📊 Promedio de clientes por AP en hora pico: {hourly_avg.loc[hourly_avg['hour']==peak_hour, 'avg_clients_per_ap'].values[0]:.1f}")


### 4.2 Visualización: Heatmap de Ocupación por Día y Hora

Muestra un heatmap que visualiza la ocupación a lo largo de la semana.


In [ ]:
# Crear pivot table para heatmap: día de la semana vs hora
heatmap_data = hourly_stats.pivot_table(
    values='total_clients',
    index='day_of_week',
    columns='hour',
    aggfunc='mean'
)

# Ordenar días de la semana correctamente
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex([day for day in day_order if day in heatmap_data.index])

# Traducir días al español (opcional)
day_names_es = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
heatmap_data.index = [day_names_es.get(day, day) for day in heatmap_data.index]

# Crear heatmap
plt.figure(figsize=(16, 8))
sns.heatmap(heatmap_data, 
            annot=True, 
            fmt='.0f', 
            cmap='YlOrRd', 
            cbar_kws={'label': 'Total Clientes'},
            linewidths=0.5,
            linecolor='gray',
            annot_kws={'size': 8})
plt.title('🔥 Heatmap de Ocupación: Total de Clientes por Día y Hora', 
          fontsize=16, fontweight='bold', pad=15)
plt.xlabel('Hora del Día', fontsize=12)
plt.ylabel('Día de la Semana', fontsize=12)
plt.tight_layout()
plt.show()

# Identificar el intervalo más ocupado
max_value = heatmap_data.max().max()
max_day = heatmap_data.max(axis=1).idxmax()
max_hour = heatmap_data.loc[max_day].idxmax()
print(f"\n🔥 Intervalo más ocupado: {max_day} a las {int(max_hour)}:00 con {max_value:,.0f} clientes")


### 4.3 Análisis por Intervalos de 15 Minutos

Análisis más detallado usando intervalos de 15 minutos para identificar patrones más granulares.


In [ ]:
# Análisis por intervalos de 15 minutos
quarterly_stats = df_aps.groupby('time_slot_15min').agg({
    'client_count': ['sum', 'mean', 'max'],
    'name': 'nunique',
    'cpu_utilization': 'mean' if 'cpu_utilization' in df_aps.columns else lambda x: 0
}).round(2)

quarterly_stats.columns = ['total_clients', 'avg_clients_per_ap', 'max_clients_single_ap', 
                           'unique_aps', 'avg_cpu_utilization']
quarterly_stats = quarterly_stats.reset_index()

# Agregar información temporal
quarterly_stats['hour'] = quarterly_stats['time_slot_15min'].dt.hour
quarterly_stats['minute'] = quarterly_stats['time_slot_15min'].dt.minute
quarterly_stats['time_str'] = quarterly_stats['time_slot_15min'].dt.strftime('%H:%M')

# Visualizar un día completo (si tenemos datos suficientes)
if len(quarterly_stats) > 0:
    # Seleccionar un día representativo (el día con más datos)
    sample_date = quarterly_stats['time_slot_15min'].dt.date.value_counts().index[0]
    daily_data = quarterly_stats[quarterly_stats['time_slot_15min'].dt.date == sample_date].copy()
    daily_data = daily_data.sort_values('time_slot_15min')
    
    # Crear etiquetas para el eje X (cada hora)
    daily_data['time_label'] = daily_data['time_slot_15min'].dt.strftime('%H:%M')
    
    fig, ax = plt.subplots(figsize=(18, 6))
    ax.plot(range(len(daily_data)), daily_data['total_clients'], 
            marker='o', linewidth=2, markersize=4, color='#8e44ad')
    ax.fill_between(range(len(daily_data)), daily_data['total_clients'], 
                     alpha=0.3, color='#9b59b6')
    ax.set_title(f'📊 Ocupación por Intervalos de 15 Minutos - {sample_date}', 
                 fontsize=16, fontweight='bold', pad=15)
    ax.set_xlabel('Hora del Día', fontsize=12)
    ax.set_ylabel('Total de Clientes', fontsize=12)
    
    # Mostrar solo algunas etiquetas para legibilidad
    step = max(1, len(daily_data) // 24)  # Aproximadamente una etiqueta por hora
    ax.set_xticks(range(0, len(daily_data), step))
    ax.set_xticklabels([daily_data.iloc[i]['time_label'] for i in range(0, len(daily_data), step)], 
                        rotation=45, ha='right')
    ax.grid(alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()
    
    print(f"\n📅 Día analizado: {sample_date}")
    print(f"📊 Intervalos de 15 minutos: {len(daily_data)}")
    print(f"🔥 Intervalo más ocupado: {daily_data.loc[daily_data['total_clients'].idxmax(), 'time_str']} "
          f"con {daily_data['total_clients'].max():,.0f} clientes")
else:
    print("⚠️  No hay suficientes datos para análisis por 15 minutos")


### 4.4 Estadísticas de Utilización de CPU (si disponible)

Si los datos incluyen información de CPU, analizamos la carga del sistema por intervalos.


In [ ]:
if 'cpu_utilization' in df_aps.columns and df_aps['cpu_utilization'].notna().sum() > 0:
    # Agrupar por hora y calcular estadísticas de CPU
    cpu_hourly = df_aps.groupby('time_slot_hour').agg({
        'cpu_utilization': ['mean', 'max', 'std'],
        'name': 'nunique'
    }).round(2)
    
    cpu_hourly.columns = ['avg_cpu', 'max_cpu', 'std_cpu', 'unique_aps']
    cpu_hourly = cpu_hourly.reset_index()
    cpu_hourly['hour'] = cpu_hourly['time_slot_hour'].dt.hour
    
    # Promedio por hora del día
    cpu_hourly_avg = cpu_hourly.groupby('hour')['avg_cpu'].mean().reset_index()
    
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(cpu_hourly_avg['hour'], cpu_hourly_avg['avg_cpu'], 
            marker='o', linewidth=2.5, markersize=8, color='#e67e22')
    ax.fill_between(cpu_hourly_avg['hour'], cpu_hourly_avg['avg_cpu'], 
                     alpha=0.3, color='#f39c12')
    ax.set_title('💻 Utilización Promedio de CPU por Hora del Día', 
                 fontsize=16, fontweight='bold', pad=15)
    ax.set_xlabel('Hora del Día', fontsize=12)
    ax.set_ylabel('Utilización de CPU (%)', fontsize=12)
    ax.set_xticks(range(0, 24))
    ax.grid(alpha=0.3, linestyle='--')
    
    # Línea de referencia al 50%
    ax.axhline(y=50, color='red', linestyle='--', linewidth=1, 
               label='Umbral 50%', alpha=0.7)
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    peak_cpu_hour = cpu_hourly_avg.loc[cpu_hourly_avg['avg_cpu'].idxmax(), 'hour']
    peak_cpu_value = cpu_hourly_avg['avg_cpu'].max()
    print(f"\n💻 Hora con mayor utilización de CPU: {int(peak_cpu_hour)}:00 ({peak_cpu_value:.1f}%)")
else:
    print("ℹ️  Información de CPU no disponible en los datos")


### 4.5 Resumen de Intervalos Más y Menos Ocupados

Identificamos los intervalos con mayor y menor ocupación.


In [ ]:
# Top 10 intervalos más ocupados
top_busy = hourly_stats.nlargest(10, 'total_clients')[
    ['time_slot_hour', 'total_clients', 'avg_clients_per_ap', 'unique_aps', 'day_of_week']
].copy()
top_busy['time_str'] = top_busy['time_slot_hour'].dt.strftime('%Y-%m-%d %H:%M')

# Top 10 intervalos menos ocupados
top_quiet = hourly_stats.nsmallest(10, 'total_clients')[
    ['time_slot_hour', 'total_clients', 'avg_clients_per_ap', 'unique_aps', 'day_of_week']
].copy()
top_quiet['time_str'] = top_quiet['time_slot_hour'].dt.strftime('%Y-%m-%d %H:%M')

print("🔥 TOP 10 INTERVALOS MÁS OCUPADOS:")
print("="*80)
for idx, row in top_busy.iterrows():
    print(f"{row['time_str']:20s} | {row['day_of_week']:10s} | "
          f"Clientes: {row['total_clients']:6,.0f} | "
          f"Promedio/AP: {row['avg_clients_per_ap']:5.1f} | "
          f"APs Activos: {row['unique_aps']:4.0f}")

print("\n" + "="*80)
print("😴 TOP 10 INTERVALOS MENOS OCUPADOS:")
print("="*80)
for idx, row in top_quiet.iterrows():
    print(f"{row['time_str']:20s} | {row['day_of_week']:10s} | "
          f"Clientes: {row['total_clients']:6,.0f} | "
          f"Promedio/AP: {row['avg_clients_per_ap']:5.1f} | "
          f"APs Activos: {row['unique_aps']:4.0f}")


ly

In [ ]:
# Clasificar días en laborables y fin de semana
hourly_stats['is_weekend'] = hourly_stats['day_of_week'].isin(['Saturday', 'Sunday'])

# Agrupar por tipo de día y hora
weekday_weekend = hourly_stats.groupby(['is_weekend', 'hour']).agg({
    'total_clients': 'mean',
    'avg_clients_per_ap': 'mean',
    'unique_aps': 'mean'
}).reset_index()

weekday_data = weekday_weekend[weekday_weekend['is_weekend'] == False]
weekend_data = weekday_weekend[weekday_weekend['is_weekend'] == True]

# Visualización comparativa
fig, ax = plt.subplots(figsize=(14, 7))
ax.plot(weekday_data['hour'], weekday_data['total_clients'], 
        marker='o', linewidth=2.5, markersize=8, color='#3498db', 
        label='Días Laborables (Lun-Vie)', alpha=0.9)
ax.fill_between(weekday_data['hour'], weekday_data['total_clients'], 
                alpha=0.2, color='#3498db')

if len(weekend_data) > 0:
    ax.plot(weekend_data['hour'], weekend_data['total_clients'], 
            marker='s', linewidth=2.5, markersize=8, color='#e74c3c', 
            label='Fin de Semana (Sáb-Dom)', alpha=0.9)
    ax.fill_between(weekend_data['hour'], weekend_data['total_clients'], 
                    alpha=0.2, color='#e74c3c')

ax.set_title('📊 Comparación de Ocupación: Días Laborables vs Fin de Semana', 
             fontsize=16, fontweight='bold', pad=15)
ax.set_xlabel('Hora del Día', fontsize=12)
ax.set_ylabel('Total de Clientes (Promedio)', fontsize=12)
ax.set_xticks(range(0, 24))
ax.grid(alpha=0.3, linestyle='--')
ax.legend(fontsize=12, loc='best')
plt.tight_layout()
plt.show()

if len(weekday_data) > 0:
    weekday_peak = weekday_data.loc[weekday_data['total_clients'].idxmax(), 'hour']
    weekday_peak_value = weekday_data['total_clients'].max()
    print(f"\n📅 Días Laborables - Hora pico: {int(weekday_peak)}:00 ({weekday_peak_value:,.0f} clientes)")

if len(weekend_data) > 0:
    weekend_peak = weekend_data.loc[weekend_data['total_clients'].idxmax(), 'hour']
    weekend_peak_value = weekend_data['total_clients'].max()
    print(f"🎉 Fin de Semana - Hora pico: {int(weekend_peak)}:00 ({weekend_peak_value:,.0f} clientes)")
    
    if len(weekday_data) > 0:
        diff = weekday_peak_value - weekend_peak_value
        pct_diff = (diff / weekday_peak_value) * 100
        print(f"📊 Diferencia: {diff:,.0f} clientes ({pct_diff:.1f}% menos en fin de semana)")


## 📈 5. Resumen y Conclusiones

### Métricas Clave Identificadas:


In [ ]:
# Calcular estadísticas generales
total_intervals = len(hourly_stats)
avg_clients_per_interval = hourly_stats['total_clients'].mean()
max_clients_interval = hourly_stats['total_clients'].max()
min_clients_interval = hourly_stats['total_clients'].min()
std_clients = hourly_stats['total_clients'].std()

print("="*70)
print("📊 RESUMEN DE ANÁLISIS DE OCUPACIÓN POR INTERVALOS")
print("="*70)
print(f"\n📅 Intervalos analizados: {total_intervals:,}")
print(f"📊 Promedio de clientes por intervalo: {avg_clients_per_interval:,.0f}")
print(f"🔥 Máximo de clientes en un intervalo: {max_clients_interval:,.0f}")
print(f"😴 Mínimo de clientes en un intervalo: {min_clients_interval:,.0f}")
print(f"📈 Desviación estándar: {std_clients:,.0f}")

# Estadísticas por hora
print(f"\n⏰ HORA PICO (promedio): {int(peak_hour)}:00")
print(f"   - Total clientes: {peak_clients:,.0f}")
print(f"   - Promedio por AP: {hourly_avg.loc[hourly_avg['hour']==peak_hour, 'avg_clients_per_ap'].values[0]:.1f}")

# Estadísticas de APs
avg_aps_per_interval = hourly_stats['unique_aps'].mean()
max_aps_per_interval = hourly_stats['unique_aps'].max()
print(f"\n📡 APs ACTIVOS:")
print(f"   - Promedio por intervalo: {avg_aps_per_interval:,.0f}")
print(f"   - Máximo en un intervalo: {max_aps_per_interval:,.0f}")

# Variabilidad
coefficient_variation = (std_clients / avg_clients_per_interval) * 100
print(f"\n📊 VARIABILIDAD:")
print(f"   - Coeficiente de variación: {coefficient_variation:.1f}%")
if coefficient_variation > 50:
    print("   ⚠️  Alta variabilidad en la ocupación")
elif coefficient_variation > 25:
    print("   ℹ️  Variabilidad moderada en la ocupación")
else:
    print("   ✅ Baja variabilidad (ocupación relativamente estable)")

print("\n" + "="*70)


## 🚀 6. Próximos Pasos y Mejoras

### Ideas para Expandir el Análisis:

1. **Análisis por Edificio/Zona:**
   - Extraer código de edificio del nombre del AP (ej: AP-VET71 → VET)
   - Comparar ocupación entre diferentes zonas del campus

2. **Análisis de Tendencias Temporales:**
   - Comparar ocupación entre diferentes meses
   - Identificar tendencias de crecimiento/decrecimiento

3. **Predicción de Carga:**
   - Usar modelos de machine learning para predecir ocupación futura
   - Identificar patrones estacionales

4. **Análisis de Capacidad:**
   - Comparar ocupación actual con capacidad máxima de los APs
   - Identificar APs sobrecargados

5. **Dashboard Interactivo:**
   - Crear visualizaciones interactivas con Plotly
   - Permitir filtros por fecha, hora, edificio, etc.

---

**🏴‍☠️ UAB THE HACK! 2025 - Análisis de Ocupación por Intervalos de Tiempo 🚀**
